## Add Indicators to Raw Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib
import os

In [ ]:
def extract_df(real_df):
    df = real_df.copy()
    # make OHLCV capitals letters
    prices = df["Close"].values
    df["SMA_20"] = talib.SMA(prices, timeperiod=20)
    df["SMA_30"] = talib.SMA(prices, timeperiod=30)
    df["SMA_50"] = talib.SMA(prices, timeperiod=50)
    df["SMA_100"] = talib.SMA(prices, timeperiod=100)
    df["SMA_200"] = talib.SMA(prices, timeperiod=200)
    df["MACD"], df["MACD_Signal"], df["MACD_Hist"] = talib.MACD(df["Close"])
    df["MACD_Crossover"] = np.where(df["MACD"] > df["MACD_Signal"], 1, -1)
    df["MACD_Crossover_Change"] = df["MACD_Crossover"].diff()
    df["RSI"] = talib.RSI(prices)
    df["MACD"], df["MACD_Signal"], df["MACD_Hist"] = talib.MACD(df["Close"])
    df["ATR"] = talib.ATR(df["High"], df["Low"], df["Close"])
    df["ADX"] = talib.ADX(df["High"], df["Low"], df["Close"])
    df["AROON_Oscillator"] = talib.AROONOSC(df["High"], df["Low"])
    df["OBV"] = talib.OBV(df["Close"], df["Volume"])
    df["CCI"] = talib.CCI(df["High"], df["Low"], df["Close"])
    df["PSAR"] = talib.SAR(df["High"], df["Low"])
    df["STOCH_K"], df["STOCH_D"] = talib.STOCH(df["High"], df["Low"], df["Close"])
    df["WILLR"] = talib.WILLR(df["High"], df["Low"], df["Close"])
    df["BBANDS_Upper"], df["BBANDS_Middle"], df["BBANDS_Lower"] = talib.BBANDS(df["Close"])
    df["AD"] = talib.AD(df["High"], df["Low"], df["Close"], df["Volume"])
    df["ADOSC"] = talib.ADOSC(df["High"], df["Low"], df["Close"], df["Volume"])
    df["VOLUME_RSI"] = talib.RSI(df["Volume"])
    df["MFI"] = talib.MFI(df["High"], df["Low"], df["Close"], df["Volume"])
    df = df.dropna()
    return df

In [ ]:

# folders_path = "/projects/genomic-ml/da2343/ml_project_2/data"
file_path = "/Users/newuser/Projects/robust-algo-trader/data/oanda"
data_file_paths = []

for csv_file in os.listdir(file_path):
    if csv_file.endswith(".csv"):
        data_file_path = os.path.join(file_path, csv_file)
        real_df = pd.read_csv(data_file_path, index_col=0, parse_dates=True)
        real_df = real_df.rename(columns={ 'time': 'Time', 
                                    'open': 'Open', 
                                    'high': 'High', 
                                    'low': 'Low', 
                                    'close': 'Close',
                                    'volume': 'Volume',
                                    })
        df = extract_df(real_df)
        file_name = os.path.basename(data_file_path)
        data_file_paths.append(file_name)
        # new_file_name = data_file_path.replace('raw_data', 'processed_data')
        
        # save the df
        # df.to_csv(new_file_name)

In [ ]:
data_file_paths

In [ ]:
df_name = 'GBP_USD_M15'
data_file_path = '/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_raw_data.csv'

real_df = pd.read_csv(data_file_path, index_col=0, parse_dates=True)
real_df = real_df.rename(columns={ 'time': 'Time', 
                                    'open': 'Open', 
                                    'high': 'High', 
                                    'low': 'Low', 
                                    'close': 'Close',
                                    'volume': 'Volume',
                                    })
df = extract_df(real_df)
df.to_csv(f"/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_processed_data.csv")

In [ ]:

for data_path in data_file_paths:
    df = extract_df(data_path)
    # remove extension from data_path
    data_path_raw = data_path.split('.')[0]
    # data_path_raw = data_path.split('/')[-1]
    new_path = data_path_raw + f'_Update.csv'
    # save the df to a csv file
    df.to_csv(new_path, index=True)

## Download Forex Data from OANDA

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
import shutil
import sys
import pandas as pd
import json

# Load the config file
# config_path = "/projects/genomic-ml/da2343/ml_project_2/settings/config.json"
config_path = "/Users/newuser/Projects/robust_algo_trader/settings/config.json"
with open(config_path) as f:
  config = json.load(f) 
config_settings = config["trading_settings"]

instrument_list = []
for key, value in config_settings.items():
    # remove _H1 from the key
    key = key.replace('_H1', '')
    instrument_list.append(key)

In [ ]:
import requests
import pandas as pd
import time


OANDA_TEST_ACCOUNT_ID
OANDA_TEST_TOKEN
time_frame = "M15"
instrument_list = ["GBP_USD"]

for instrument in instrument_list: 
    start_date = "2006-08-14T20:00:00.000000000Z"
    candle_length = 1000
    forex_data_list = []
    while candle_length > 0:
        url = f"https://api-fxpractice.oanda.com/v3/accounts/{OANDA_TEST_ACCOUNT_ID}/instruments/{instrument}/candles?granularity={time_frame}&from={start_date}&includeFirst=False"
        headers = {"Authorization": f"Bearer {OANDA_TEST_TOKEN}"}
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            candles = data["candles"]
            candle_length = len(candles)
            
            if candle_length == 0:
                print(f"Number of candles: {candle_length}")
                break
            
            start_date = candles[-1]["time"]
            for candle in candles:
                local_candle = {
                    "time": candle["time"],
                    "open": candle["mid"]["o"],
                    "high": candle["mid"]["h"],
                    "low": candle["mid"]["l"],
                    "close": candle["mid"]["c"],
                    "volume": candle["volume"]
                }
                forex_data_list.append(local_candle)
            # print(f"Start Date: {start_date}")
            # break
            time.sleep(5)
        else:
            print(f"Error: {response.status_code}")
            break
        
    forex_data = pd.DataFrame(forex_data_list)
    forex_data.to_csv(f"{instrument}_{time_frame}_raw_data.csv", index=False)
    

In [5]:
forex_data = pd.DataFrame(forex_data_list)
# forex_data.to_csv(f"{instrument}_{time_frame}_raw_data.csv", index=False)
forex_data


,time,open,high,low,close,volume
0,2006-08-14T20:01:00.000000000Z,1.88775,1.88780,1.88775,1.88775,3
1,2006-08-14T20:02:00.000000000Z,1.88775,1.88775,1.88775,1.88775,3
2,2006-08-14T20:03:00.000000000Z,1.88775,1.88775,1.88760,1.88760,5
3,2006-08-14T20:04:00.000000000Z,1.88765,1.88770,1.88756,1.88770,8
4,2006-08-14T20:05:00.000000000Z,1.88770,1.88775,1.88765,1.88775,4
...,...,...,...,...,...,...
371251,2007-08-15T04:55:00.000000000Z,1.98844,1.98891,1.98844,1.98876,35
371252,2007-08-15T04:56:00.000000000Z,1.98879,1.98936,1.98879,1.98908,37
371253,2007-08-15T04:57:00.000000000Z,1.98911,1.98971,1.98911,1.98952,36
371254,2007-08-15T04:58:00.000000000Z,1.98959,1.98986,1.98946,1.98954,21
